# Tensor  
PyTorch 的官方介绍是一个拥有强力GPU加速的张量和动态构建网络的库，其主要构件是张量，所以我们可以把 PyTorch 当做 NumPy 来用，PyTorch 的很多操作好 NumPy 都是类似的，但是因为其能够在 GPU 上运行，所以有着比 NumPy 快很多倍的速度   

关于张量的本质不乏深度的剖析，但从工程角度来讲，可简单地认为它就是一个数组，且支持高效的科学计算。它可以是一个数（标量）、一维数组（向量）、二维数组（矩阵）和更高维的数组（高阶数据）。Tensor和Numpy的ndarrays类似，但PyTorch的tensor支持GPU加速

In [0]:
import torch as t
t.__version__

'1.1.0'

## 1.1 Tensor类型   
默认的tensor是FloatTensor，可通过`t.set_default_tensor_type` 来修改默认tensor类型。Tensor有不同的数据类型，每种类型分别对应有CPU和GPU版本(HalfTensor除外)。      
`HalfTensor`是专门为GPU版本设计的，同样的元素个数，显存占用只有FloatTensor的一半，所以可以极大缓解GPU显存不足的问题，但由于HalfTensor所能表示的数值大小和精度有限，所以可能出现溢出等问题。    

| Data type                | dtype                             | CPU tensor                                                   | GPU tensor                |
| ------------------------ | --------------------------------- | ------------------------------------------------------------ | ------------------------- |   
| 32-bit floating point    | `torch.float32` or `torch.float`  | `torch.FloatTensor`                                          | `torch.cuda.FloatTensor`  |
| 64-bit floating point    | `torch.float64` or `torch.double` | `torch.DoubleTensor`                                         | `torch.cuda.DoubleTensor` |
| 16-bit floating point    | `torch.float16` or `torch.half`   | `torch.HalfTensor`                                           | `torch.cuda.HalfTensor`   |
| 8-bit integer (unsigned) | `torch.uint8`                     | `torch.ByteTensor` | `torch.cuda.ByteTensor`   |
| 8-bit integer (signed)   | `torch.int8`                      | `torch.CharTensor`                                           | `torch.cuda.CharTensor`   |
| 16-bit integer (signed)  | `torch.int16` or `torch.short`    | `torch.ShortTensor`                                          | `torch.cuda.ShortTensor`  |
| 32-bit integer (signed)  | `torch.int32` or `torch.int`      | `torch.IntTensor`                                            | `torch.cuda.IntTensor`    |
| 64-bit integer (signed)  | `torch.int64` or `torch.long`     | `torch.LongTensor`                                           | `torch.cuda.LongTensor`   |      
    
 - CPU tensor与GPU tensor之间的互相转换通过tensor.cuda和tensor.cpu方法实现，此外还可以使用tensor.to(device)    
 - 各数据类型之间可以互相转换，常见用法type(new_type)，同时还有float、long、half等快捷方法
 - Tensor还有`new`方法，用法与`t.Tensor`一样，会调用该tensor对应类型的构造函数，生成与当前tensor类型一致的tensor  
 - `torch.*_like(a)`可以生成和`a`拥有同样属性(类型，形状，cpu/gpu)的新tensor   
 - `tensor.new_*(new_shape)` 新建一个不同形状的tensor

In [0]:
a = t.Tensor(3, 4)
a.dtype 

torch.float32

In [0]:
t.zeros_like(a)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [0]:
a.new(2, 3)

tensor([[8.4487e-37, 0.0000e+00, 3.7835e-44],
        [0.0000e+00,        nan, 0.0000e+00]])

In [0]:
a.new_tensor([2, 3])

tensor([2., 3.])

### 逐元素操作     
对tensor的每一个元素(point-wise，又名element-wise)进行操作，此类操作的输入与输出形状一致        


|函数|功能|
|:--:|:--:|
|abs/sqrt/div/exp/fmod/log/pow..|绝对值/平方根/除法/指数/求余/求幂..|
|cos/sin/asin/atan2/cosh..|相关三角函数|
|ceil/round/floor/trunc| 上取整/四舍五入/下取整/只保留整数部分|
|clamp(input, min, max)|超过min和max部分截断|
|sigmod/tanh..|激活函数    
    
其中`clamp(x, min, max)`的输出满足以下公式：
$$
y_i =
\begin{cases}
min,  & \text{if  } x_i \lt min \\
x_i,  & \text{if  } min \le x_i \le max  \\
max,  & \text{if  } x_i \gt max\\
\end{cases}
$$
`clamp`常用在某些需要比较大小的地方，如取一个tensor的每个元素与另一个数的较大值

In [0]:
a = t.arange(0, 12).view(3, 4)
a ** 2

tensor([[  0,   1,   4,   9],
        [ 16,  25,  36,  49],
        [ 64,  81, 100, 121]])

In [0]:
t.clamp(a, min=3, max=9)

tensor([[3, 3, 3, 3],
        [4, 5, 6, 7],
        [8, 9, 9, 9]])

###  归并操作 
此类操作会使输出形状小于输入形状，并可以沿着某一维度进行指定操作。如加法`sum`，既可以计算整个tensor的和，也可以计算tensor中每一行或每一列的和。    

|函数|功能|
|:---:|:---:|
|mean/sum/median/mode|均值/和/中位数/众数|
|norm/dist|范数/距离|
|std/var|标准差/方差|
|cumsum/cumprod|累加/累乘|

以上大多数函数都有一个参数**`dim`**，用来指定这些操作是在哪个维度上执行的。关于dim(对应于Numpy中的axis)的解释众说纷纭，这里提供一个简单的记忆方式：

假设输入的形状是(m, n, k)

- 如果指定dim=0，输出的形状就是(1, n, k)或者(n, k)
- 如果指定dim=1，输出的形状就是(m, 1, k)或者(m, k)
- 如果指定dim=2，输出的形状就是(m, n, 1)或者(m, n)

size中是否有"1"，取决于参数`keepdim`，`keepdim=True`会保留维度`1`。注意，以上只是经验总结，并非所有函数都符合这种形状变化方式，如`cumsum`。

In [0]:
a = t.ones(2, 3)
print(a, a.shape)

tensor([[1., 1., 1.],
        [1., 1., 1.]]) torch.Size([2, 3])


In [0]:
b = a.sum(dim=0, keepdim=True)
b, b.shape

(tensor([[2., 2., 2.]]), torch.Size([1, 3]))

In [0]:
b = a.sum(dim=0, keepdim=False)
b, b.shape

(tensor([2., 2., 2.]), torch.Size([3]))

In [0]:
c = a.sum(dim=1)
c, c.shape

(tensor([3., 3.]), torch.Size([2]))

In [0]:
c = a.sum(dim=1, keepdim=True)
c, c.shape

(tensor([[3.],
         [3.]]), torch.Size([2, 1]))

In [0]:
# 按行累加
t.cumsum(a, dim=1)

tensor([[1., 2., 3.],
        [1., 2., 3.]])

### 比较
比较函数中有一些是逐元素比较，操作类似于逐元素操作，还有一些则类似于归并操作     

|函数|功能|
|:--:|:--:|
|gt/lt/ge/le/eq/ne|大于/小于/大于等于/小于等于/等于/不等|
|topk|最大的k个数|
|sort|排序|
|max/min|比较两个tensor最大最小值|

表中第一行的比较操作已经实现了运算符重载，因此可以使用`a>=b`、`a>b`、`a!=b`、`a==b`，其返回结果是一个`ByteTensor`，可用来选取元素。max/min这两个操作比较特殊，以max来说，它有以下三种使用情况：
- t.max(tensor)：返回tensor中最大的一个数
- t.max(tensor,dim)：指定维上最大的数，返回tensor和下标
- t.max(tensor1, tensor2): 比较两个tensor相比较大的元素

In [0]:
a = t.linspace(0, 15, 6).view(2, 3)
a

tensor([[ 0.,  3.,  6.],
        [ 9., 12., 15.]])

In [0]:
b = t.linspace(15, 0, 6).view(2, 3)
b

tensor([[15., 12.,  9.],
        [ 6.,  3.,  0.]])

In [0]:
a > b

tensor([[0, 0, 0],
        [1, 1, 1]], dtype=torch.uint8)

In [0]:
# 第一个返回值表示每行最大值，第二个返回值表示 其索引 
t.max(a, dim=1)

torch.return_types.max(values=tensor([ 6., 15.]), indices=tensor([2, 2]))

### 线性代数     
PyTorch的线性函数主要封装了Blas和Lapack，其用法和接口都与之类似     

|函数|功能|
|:---:|:---:|
|trace|对角线元素之和(矩阵的迹)|
|diag|对角线元素|
|triu/tril|矩阵的上三角/下三角，可指定偏移量|
|mm/bmm|矩阵乘法，batch的矩阵乘法|
|addmm/addbmm/addmv/addr/badbmm..|矩阵运算
|t|转置|
|dot/cross|内积/外积
|inverse|求逆矩阵
|svd|奇异值分解

需要注意的是，矩阵的转置会导致存储空间不连续，需调用它的`.contiguous`方法将其转为连续

In [0]:
b = a.t()
b.is_contiguous()

False

In [0]:
b.contiguous()

tensor([[ 0.,  9.],
        [ 3., 12.],
        [ 6., 15.]])

## 1.2 Tensor 和 Numpy      
Tensor和Numpy数组之间具有很高的相似性，彼此之间的互操作也非常简单高效。要注意的是，**Numpy和Tensor共享内存**    
由于Numpy历史悠久，支持丰富的操作，所以当遇到Tensor不支持的操作时，可先转成Numpy数组，处理后再转回tensor，其转换开销很小

In [0]:
import numpy as np

a = np.ones([2, 3])
a.dtype

dtype('float64')

In [0]:
# numpy --> tensor
b = t.from_numpy(a)

c = t.Tensor(a)

print(b)
print(c, c.shape)

tensor([[1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[1., 1., 1.],
        [1., 1., 1.]]) torch.Size([2, 3])


In [0]:
a[0, 0] = 123
b

tensor([[123.,   1.,   1.],
        [  1.,   1.,   1.]], dtype=torch.float64)

In [0]:
d = b.numpy()
d

array([[123.,   1.,   1.],
       [  1.,   1.,   1.]])

**注意**： 当numpy的数据类型和Tensor的类型不一样的时候，数据会被复制，不会共享内存。

In [0]:
a = np.ones([2, 3])
a.dtype

dtype('float64')

In [0]:
# 此处进行拷贝，不共享内存
b = t.Tensor(a)
b.dtype

torch.float32

In [0]:
a[0, 0] = 123
b

tensor([[1., 1., 1.],
        [1., 1., 1.]])

**注意：** 不论输入的类型是什么，`t.tensor`都会进行数据拷贝，不会共享内存

In [0]:
c = t.tensor(a)
c

tensor([[123.,   1.,   1.],
        [  1.,   1.,   1.]], dtype=torch.float64)

In [0]:
c[0, 0] = 0
a

array([[123.,   1.,   1.],
       [  1.,   1.,   1.]])

### 广播规则   
广播法则(broadcast)是科学运算中经常使用的一个技巧，它在快速执行向量化的同时不会占用额外的内存/显存。
Numpy的广播法则定义如下：

- 让所有输入数组都向其中shape最长的数组看齐，shape中不足的部分通过在前面加1补齐
- 两个数组要么在某一个维度的长度一致，要么其中一个为1，否则不能计算 
- 当输入数组的某个维度的长度为1时，计算时沿此维度复制扩充成一样的形状

PyTorch当前已经支持了自动广播法则，但是笔者还是建议读者通过以下两个函数的组合手动实现广播法则，这样更直观，更不易出错：

- `unsqueeze`或者`view`，或者tensor[None],：为数据某一维的形状补1，实现法则1
- `expand`或者`expand_as`，重复数组，实现法则3；该操作不会复制数组，所以不会占用额外的空间。

注意，repeat实现与expand相类似的功能，但是repeat会把相同数据复制多份，因此会占用额外的空间

In [0]:
a = t.ones(3, 2)
b = t.zeros(2, 3, 1)
print(a)
print(b)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[[0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.]]])


In [0]:
# 自动广播步骤：
# 第一步：a是2维数组,b是3维数组，先在较小的a前面补1 维，
#               即：a.unsqueeze(0)，a的形状变成（1，3，2），b的形状是（2，3，1)
# 第二步：a和b在第一维和第三维形状不一样，其中一个为1 ，
#               可以利用广播法则扩展，两个形状都变成了（2，3，2）
c = a + b
c, c.shape

(tensor([[[1., 1.],
          [1., 1.],
          [1., 1.]],
 
         [[1., 1.],
          [1., 1.],
          [1., 1.]]]), torch.Size([2, 3, 2]))

In [0]:
# expand不会占用额外空间，只有需要时才扩充，极大的节省了内存
a[None].expand(2, 3, 2) + b.expand(2, 3, 2)
#a.view(1,3,2).expand(2,3,2)+b.expand(2,3,2)

tensor([[[1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.]]])

## 1.3 tensor内部结构    
tensor分为头信息区(Tensor)和存储区(Storage)，信息区主要保存着tensor的形状（size）、步长（stride）、数据类型（type）等信息，而真正的数据则保存成连续数组。      
由于数据动辄成千上万，因此信息区元素占用内存较少，主要内存占用则取决于tensor中元素的数目，也即存储区的大小。

一般来说一个tensor有着与之相对应的storage, storage是在data之上封装的接口，便于使用，而不同tensor的头信息一般不同，但却可能使用相同的数据

In [0]:
a = t.arange(0, 6)
a.storage()

 0
 1
 2
 3
 4
 5
[torch.LongStorage of size 6]

In [0]:
b = a.view(2, 3)
b.storage()

 0
 1
 2
 3
 4
 5
[torch.LongStorage of size 6]

In [0]:
# id可看作对象在内存中的地址，ab共享storage
id(a.storage()) == id(b.storage())

True

In [0]:
a[0] = 123
b

tensor([[123,   1,   2],
        [  3,   4,   5]])

In [0]:
a.data_ptr(), b.data_ptr()

(44393664, 44393664)